In [7]:
# PDF Loaders. If unstructured gives you a hard time, try PyPDFLoader
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_pinecone import PineconeVectorStore
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
import os
load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')

# loader = PyPDFLoader('./charlie-and-the-chocolate-factory-by-roald-dahl.pdf')
loader = PyPDFLoader('./wentiejun.pdf')
pages = loader.load()
text = "".join(page.page_content for page in pages)
text = text.replace('\t', ' ')
# print(text)
print(pages[1])
# Note: If you're using PyPDFLoader then it will split by page for you already
print (f'You have {len(pages)} document(s) in your data')
# print (f'There are {len(pages[1].page_content)} characters in your sample document')
# print (f'Here is a sample: {pages[1].page_content[:200]}')

# We'll split our data into chunks around 500 characters each with a 50 character overlap. These are relatively small.
# text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", "\t"], chunk_size=10000, chunk_overlap=3000)
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", "\t"], chunk_size=4000, chunk_overlap=200)
docs = text_splitter.create_documents([text])
# Let's see how many small chunks we have
print (f'Now you have {len(docs)} documents')
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
# vectors = embeddings.embed_documents([x.page_content for x in docs])

page_content='八次危机八次危机\nEIGHT CRISES EIGHT CRISES\n中国的真实经验 1949-2009中国的真实经验 1949-2009\nLESSONS FROM CHINA，1949-2009LESSONS FROM CHINA，1949-2009\n温铁军 等著\n东 方 出 版 社\n2 0 1 3 年 1 月' metadata={'source': './wentiejun.pdf', 'page': 1}
You have 480 document(s) in your data
Now you have 75 documents


In [14]:
print(docs[0])

page_content='八次危机八次危机\nEIGHT CRISES EIGHT CRISES\n中国的真实经验 1949-2009中国的真实经验 1949-2009\nLESSONS FROM CHINA，1949-2009LESSONS FROM CHINA，1949-2009\n温铁军 等著\n东 方 出 版 社\n2 0 1 3 年 1 月图 书 在 版 编 目 （ C I P ） 数 据\n八次危机/温铁军 等著. —北京：东方出版社，2012.10\nISBN 978-7-5060-5557-4\nⅠ.①八… Ⅱ.①温… Ⅲ.①中国经济—经济危机—研究\nⅣ.①F124.8\n中国版本图书馆CIP数据核字（2012）第 254505 号\n八次危机：中国的真实经验 1949—2009\n（BACI WEIJI:ZHONGGUO DE ZHENSHI JINGYAN 1949-2009）\n作       者：温铁军 等\n责任编辑：徐玲\xa0龚雪\n出       版：东方出版社\n发       行：人民东方出版传媒有限公司\n地       址：北京市东城区朝阳门内大街166号\n邮政编码：100706\n印       刷：北京市大兴县新魏印刷厂版       次：2013年1月第1版\n印       次：2013年1月第2次印刷\n印       数：10001—15000册\n开       本：710毫米×1000毫米 1/16\n印       张：22\n字       数：305千字\n书       号：ISBN 978-7-5060-5557-4\n定       价：55.00元\n发行电话：（010）65210056 65210060 65210062 65210063\n版权所有，违者必究 本书观点并不代表本社立场\n如有印刷质量问题，请拨打电话：（010）65210012\xa0\n\xa0\n\xa0\n天行有常。不为尧存，不为桀亡。\n循道而不贰，则天不能祸。\n——苟子《天论》致 谢\n本书辑录的科研成果得到以下支持，特此鸣谢：\n中国国家社会科学基金会重大项目、重点项目\n中国教育部人文社会科学重大攻关项目、应急项目\n中国国家自然科学基金应急项目\n北京市交叉学科重点项目，首都经济学科群建设项目\n中国人民

In [15]:

os.environ['PINECONE_API_KEY'] = os.getenv('PINECONE_API_KEY')
index_name = 'hkpineconeindex1'
vectorstore_from_texts = PineconeVectorStore.from_texts(
    [x.page_content for x in docs],
    index_name=index_name,
    embedding=embeddings
)
# query='中国经济现阶段是怎么样的？'
query='哪一次危机影响最严重'
docs2 = vectorstore_from_texts.similarity_search(query, 
    k=4  # return 4 most relevant docs 
)
print(len(docs2))


4


In [17]:
print(len(docs2[0].page_content))

3975


In [18]:
import tiktoken
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens
count = num_tokens_from_string(docs2[0].page_content, "cl100k_base")
print(count)

4217


In [19]:
llm = ChatOpenAI(temperature=0, openai_api_key=os.getenv('OPENAI_API_KEY'), model='gpt-3.5-turbo-16k') 
# https://platform.openai.com/docs/models/gpt-3-5-turbo  token limit
chain = load_qa_chain(llm, chain_type="stuff")
chain.run(input_documents=docs2, question=query)
# chain.run(input_documents=[docs2[1]], question=query)

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 18033 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}